## Remplissage du dataset des pit-stops

Le dataset que nous utilisons est incomplet : en effet, il ne propose les informations de pit-stop qu'à partir de la 841e course, alors que les données des temps au tour sont disponibles pour beaucoup plus de courses. Nous souhaitons remplir ce dataset.

On remarque qu'un pit-stop est symbolisé sur le graphique des temps au tour par un pic brusque entre deux tours rapides. Il est donc possible d'utiliser un algorithme d'apprentissage supervisé pour trouver les pit-stops à chaque course.

![herothisaustria.jpg](attachment:herothisaustria.jpg)

In [ ]:
import os
import tempfile
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

#import scikitplot as skplt
import warnings
warnings.filterwarnings("ignore")

sns.set()
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


In [ ]:
pitStops = pd.read_csv("datasets/pit_stops.csv")
lapTimes = pd.read_csv("datasets/lap_times.csv")
races = pd.read_csv("datasets/races.csv",index_col=0)
races = races.drop(columns=['url','time'])

### Préparation des sets : 
on va avoir un training set d'environ beaucoup d'exemples. On peut essayer d'appliquer la technique de cross-validation pour entraîner le modèle. <br>
Quelles sont les features ? 
<ul>
    <li>driverId</li>
    <li>circuitId</li>
    <li>lap</li>
    <li>milliseconds</li>
</ul>
Le label est un vecteur y de taille $m = nbLaps$.

In [ ]:
#dataset_pit.csv a été généré par moi, il contient les courses dont on connaît les informations de pit
my_dataset = pd.read_csv('dataset_pit.csv', index_col=0)
#En raison d'incidents pouvant survenir en début de course, on a beaucoup de pits dans les premiers tours
#Ils créent un bruit inutile et peuvent rendre la prédiction moins précise
#On retire donc les informations de tour avant le 5e tour.
my_dataset = my_dataset[my_dataset['lap'] > 5]

#On va créer une colonne contenant le temps du tour précédent
my_dataset['previousLap'] = np.zeros(my_dataset.shape[0])

#Pour ce faire, on effectue une translation du dataset dans une copie.
my_dataset_trans = my_dataset.shift(1)

#On duplique la première ligne pour qu'elle ne soit pas vide
my_dataset_trans.iloc[0] = my_dataset.iloc[0]

#On associe les deux champs. On rajoute raceId et driverId pour que le tour précédent corresponde bien au bon pilote.
my_dataset['previousLap'] = my_dataset_trans['milliseconds']
my_dataset['previousLapRaceId'] = my_dataset_trans['raceId']
my_dataset['previousLapDriverId'] = my_dataset_trans['driverId']
my_dataset['previousLapPit'] = my_dataset_trans['pit']

#Le rapport du temps au tour actuel et du temps au tour précédent.
my_dataset['previousLapDelta'] = np.where((my_dataset['raceId'] == my_dataset['previousLapRaceId']) & (my_dataset['driverId'] == my_dataset['previousLapDriverId']) & my_dataset['previousLapPit'] != 1, my_dataset['milliseconds']/my_dataset['previousLap'], 1)

#On répète exactement la même opération pour le tour suivant
my_dataset_fwd = my_dataset.shift(-1)
my_dataset_fwd.loc[my_dataset_fwd.shape[0]] =  my_dataset.loc[my_dataset.shape[0]]
my_dataset['nextLap'] = my_dataset_fwd['milliseconds']
my_dataset['nextLapRaceId'] = my_dataset_fwd['raceId']
my_dataset['nextLapDriverId'] = my_dataset_fwd['driverId']
my_dataset['nextLapPit'] = my_dataset_fwd['pit']
my_dataset['nextLapDelta'] = np.where((my_dataset['raceId'] == my_dataset['nextLapRaceId']) & (my_dataset['driverId'] == my_dataset['nextLapDriverId']) & my_dataset['nextLapPit'] != 1, (my_dataset['milliseconds']/my_dataset['nextLap']), 1)

#lapDelta correspond à la différence du tour actuel par rapport au précédent ET au suivant
my_dataset['lapDelta'] = (my_dataset['previousLapDelta']+my_dataset['nextLapDelta'])/2

#On accentue les résultats : si lapDelta ~ 1 alors on affecte une valeur 1-exp(1-1) ~ 0 
#                            si lapDelta >> 1 alors on affecte une valeur 10*(1-exp(1-1,5)) >~ 1
my_dataset['lapDelta'] = my_dataset['lapDelta'].apply(lambda x: 10.0*(1-np.exp(1-x)))

#On centre les résultats (attention : l'algorithme fonctionne mieux si on ne le fait pas)
# my_dataset['lapDelta'] = (my_dataset['lapDelta']-my_dataset['lapDelta'].mean())/my_dataset['lapDelta'].std()

#Si la valeur est plus petite que 0,05 on affecte 0, sinon on multiplie par 10
#"Feature engineering at its finest"
my_dataset['lapDelta'] = np.where((np.abs(my_dataset['lapDelta']) < 0.05) | (my_dataset['pit'] == 0),0,10*my_dataset['lapDelta'])

#On retire les colonnes dont on n'a pas besoin
my_dataset = my_dataset.drop(['previousLap','previousLapRaceId','previousLapDriverId','nextLap','nextLapRaceId','nextLapDriverId'],axis=1)

#On retire les 50 valeurs les plus grandes qui peuvent empêcher un apprentissage de qualité ta3 Montessori
for i in range(1,50):
    my_dataset = my_dataset.drop(my_dataset['previousLapDelta'].idxmax())
    my_dataset = my_dataset.drop(my_dataset['nextLapDelta'].idxmax())

#On affiche le dataset préparé.    
display(my_dataset.head(10))

On peut transformer le problème en un problème de régression logistique : <b>A un tour donné, quelle est la probablilité qu'un pilote réalise un arrêt au stand ?

On peut remarquer ici que seuls 4% des labels représentent un arrêt au stand : les données ne sont pas du tout équitablement réparties !!

In [ ]:
neg, pos = np.bincount(my_dataset['pit'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({}% of total)\n'.format(
    total, pos, 100 * pos / total))

On crée les ensembles d'entraînement, de validation et de test.

In [ ]:
#On tente de prédire avec une seule feature mais qui est la combinaison de plusieurs autres
cleaned_df = my_dataset[['lapDelta','pit']]

#On retire les valeurs nulles
cleaned_df = cleaned_df.dropna()

#On utilise une fonction de sklearn pour créer les ensembles d'entraînement et de test.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

#On extrait les labels et les données
train_labels = np.array(train_df.pop('pit'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('pit'))
test_labels = np.array(test_df.pop('pit'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

#On sépare le dataset pour pouvoir affecter des poids de classe : ceci est important pour contrebalancer
#le fait que les classes sont inéquitablement réparties
"""ATTENTION : PAS ENCORE IMPLEMENTE DANS L'ALGORITHME DE REGRESSION"""
pit_y = cleaned_df[cleaned_df['pit'] == 1]
pit_n = cleaned_df[cleaned_df['pit'] == 0]
neg = pit_n["pit"].count()
pos = pit_y['pit'].count()
total = neg+pos
display(pit_y.describe())
display(pit_n.describe())
#On divise par 2 pour garder la perte dans le même ordre de grandeur
weight_for_0 = (1.0 / neg)*(total)/2.0 
weight_for_1 = (1.0 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
from logistic import LogisticClassifier
classifier = LogisticClassifier()
model = classifier.fit(train_features.T, train_labels,num_iterations = 2000, learning_rate = 0.05, print_cost = True)

In [ ]:
#On récupère l'historique des coûts
costH = model['costs']
#On récupère les prédictions
predictions = classifier.predict(test_features.T)
test_labels = np.transpose(test_labels)
# print(predictions.tolist())
print(np.sum(predictions))
print(test_labels.shape)
print(predictions[predictions>0.2] )

In [ ]:
print(len(costH))
fig = plt.figure(figsize=(8,6))
ax3 = fig.add_subplot(111)
l1, = ax3.plot(np.arange(len(costH)), costH, 'red')
plt.suptitle('Evolution de la RMSE en fonction des iterations')
plt.yscale('log')


On affiche la matrice de confusion

**TODO : Coder la matrice à la main**

In [ ]:
def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Vrai label')
    plt.xlabel('Label predit')

plot_cm(test_labels[0], predictions[0])